In [1]:
import keras
import pandas as pd
import tensorflow as tf
from keras.layers import *
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
df = pd.read_csv("train_for_predict_adr.csv")
df = df.drop(["date"], axis=1)
df.head()

,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,is_repeated_guest,previous_cancellations,...,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,1,0,257,27,0,2,1,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
1,1,0,257,27,0,2,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
2,1,0,257,27,0,2,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
3,1,0,257,27,0,2,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
4,1,0,257,27,0,2,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [3]:
X = df.drop(["adr"], axis=1).values
y = df["adr"].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1126)

In [5]:
earlystop = EarlyStopping(monitor='val_loss', patience=10)
reducelr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=0.001)

In [6]:
model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, kernel_initializer='glorot_uniform', activation='linear'))

model.compile(loss='mean_squared_error', optimizer='nadam')

In [7]:
model.fit(X_train, y_train, epochs=500, batch_size=128, validation_split=0.2, callbacks=[earlystop, reducelr])

Epoch 1/500
286/286 [==============================] - 1s 3ms/step - loss: 2479.4707 - val_loss: 1172.6636
Epoch 2/500
286/286 [==============================] - 1s 2ms/step - loss: 1073.1564 - val_loss: 898.6708
Epoch 3/500
286/286 [==============================] - 1s 2ms/step - loss: 928.5464 - val_loss: 820.7605
Epoch 4/500
286/286 [==============================] - 1s 2ms/step - loss: 864.0251 - val_loss: 777.2284
Epoch 5/500
286/286 [==============================] - 1s 2ms/step - loss: 813.8376 - val_loss: 738.9531
Epoch 6/500
286/286 [==============================] - 1s 2ms/step - loss: 771.3060 - val_loss: 723.0679
Epoch 7/500
286/286 [==============================] - 1s 2ms/step - loss: 736.2095 - val_loss: 670.4711
Epoch 8/500
286/286 [==============================] - 1s 2ms/step - loss: 712.6292 - val_loss: 648.5619
Epoch 9/500
286/286 [==============================] - 1s 2ms/step - loss: 689.9888 - val_loss: 636.6264
Epoch 10/500
286/286 [==============================

286/286 [==============================] - 1s 2ms/step - loss: 479.1942 - val_loss: 457.5146
Epoch 79/500
286/286 [==============================] - 1s 2ms/step - loss: 474.3327 - val_loss: 454.1947
Epoch 80/500
286/286 [==============================] - 1s 2ms/step - loss: 475.1895 - val_loss: 463.9359
Epoch 81/500
286/286 [==============================] - 1s 2ms/step - loss: 472.6957 - val_loss: 456.9500
Epoch 82/500
286/286 [==============================] - 1s 2ms/step - loss: 476.3973 - val_loss: 463.2870
Epoch 83/500
286/286 [==============================] - 1s 2ms/step - loss: 471.3828 - val_loss: 471.2455
Epoch 84/500
286/286 [==============================] - 1s 2ms/step - loss: 471.6546 - val_loss: 458.5663
Epoch 85/500
286/286 [==============================] - 1s 2ms/step - loss: 472.4332 - val_loss: 461.1533
Epoch 86/500
286/286 [==============================] - 1s 2ms/step - loss: 466.8271 - val_loss: 451.7916
Epoch 87/500
286/286 [==============================] - 1s 

In [8]:
model.evaluate(X_test, y_test)

358/358 [==============================] - 0s 1ms/step - loss: 432.5131


432.51312255859375

In [17]:
# model.save("adr_model.h5")

In [ ]:
model.summary()

In [ ]:
model = keras.models.load_model("adr_model.h5")

In [59]:
y_pred = pd.DataFrame(model.predict(X), columns=["adr"])
y_pred.shape

(57172, 1)

In [60]:
y_pred.to_csv("train_adr_nn.csv", index=False)

### Predict for test data ADR

In [52]:
df = pd.read_csv("test_for_predict_adr.csv")
df = df.drop(["date", "adr"], axis=1)
df.head()

,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,is_repeated_guest,previous_cancellations,...,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,1,0,75,13,2,5,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
1,0,0,12,13,2,5,2,0.0,0,0,...,0,1,0,1,0,0,1,0,0,0
2,1,0,174,13,2,2,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
3,1,0,36,13,1,1,1,2.0,0,0,...,0,1,0,1,0,0,0,0,1,0
4,1,0,0,13,1,1,2,1.0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [53]:
X = df.values
y_pred = pd.DataFrame(model.predict(X), columns=["adr"])

In [54]:
y_pred.to_csv("test_adr_nn.csv", index=False)